In [15]:
import os
import json

import pandas as pd
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from itertools import chain

from comparison.wasserstein import emd

In [16]:
import matplotlib as mpl
import matplotlib.font_manager as font_manager
from matplotlib import rcParams

#font_path = '/home/andyclee/.fonts/LinLibertine_R.ttf'  # Your font path goes here
#font_manager.fontManager.addfont(font_path)
#prop = font_manager.FontProperties(fname=font_path)

#mpl.rcParams['font.family'] = 'Linux Libertine'

In [17]:
def calc_dists(arr, attr_dict, kappa):

    opairs = []

    max_edges = (kappa - 1) * (kappa - 2) / 2
    three_walks = (arr @ arr) @ arr
    for v, vtype in attr_dict.items():
        deg = sum(arr[v])

        assort = 0
        for u, con in enumerate(arr[v]):
            if con and attr_dict[u] == attr_dict[v]:
                assort += 1
            elif con and attr_dict[u] != attr_dict[v]:
                assort -= 1

        if deg == 0:
            assort = 0
        else:
            assort = assort / deg

        ccoeff = 0
        if deg <= 1:
            ccoeff = 0
        else:
            ccoeff = (three_walks[v][v] / 2) / (deg * (deg - 1))

        opairs.append( ( (assort + 1) / 2, ccoeff) )

    dist_map = {}
    for op in opairs:
        if op in dist_map:
            dist_map[op] += 1
        else:
            dist_map[op] = 1

    dist = []
    for op, freq in dist_map.items():
        dist.append( (op, freq / len(attr_dict)) )

    return dist

def avg_dists(dists):
    dist_freq = {}
    for d in dists:
        for op, freq in d:
            if op in dist_freq:
                dist_freq[op] += freq
            else:
                dist_freq[op] = freq
    for op, freq in dist_freq.items():
        dist_freq[op] = freq / len(dists)

    return [ (op, freq) for op, freq in dist_freq.items() ]

In [18]:
# try a new metric that has global and local
def triangle_count(adj_mat):
    return np.trace(np.linalg.matrix_power(adj_mat, 3)) / 6

def assortativity_coeff(adj_mat, types):
    n = len(adj_mat)
    
    # 2 times the edge count, based on paper normalization
    m = np.sum(adj_mat)
    e = np.zeros((2, 2))
    for i in range(n):
        for j in range(n):
            if adj_mat[i][j] != 1:
                continue
            
            if types[i] == types[j] and types[i] == -1:
                e[0][0] += 1
            elif types[i] != types[j] and types[i] == -1:
                e[0][1] += 1
            elif types[i] != types[j] and types[i] == 1:
                e[1][0] += 1
            elif types[i] == types[j] and types[i] == 1:
                e[1][1] += 1
    
    e = e / m
    e_square = np.square(e)
    es_sum = np.sum(e_square)
    e_trace = np.trace(e)
    assort = (e_trace - es_sum) / (1 - es_sum)
    return (assort + 1) / 2

def ntwk_metric(dist1, tc1, assort1, dist2, tc2, assort2):
    distr_metric = emd(dist1, dist2)
    
    tc_diff = abs(tc1 - tc2) / (tc1 + tc2) # https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error
    assort_diff = abs(assort1 - assort2) / (assort1 + assort2)
    global_diff = (tc_diff + assort_diff) # don't divide by 2 because distributional metric is between [0, 2]
    
    return (distr_metric + global_diff) / 2

In [19]:
vill_list_old = chain(range(12),range(13, 21),range(22, 77))

vill_list = [x+1 for x in vill_list_old]

#vill_list = [10]

sc_old = range(17)
ho_old = range(17)

sc_list = [x/16 for x in sc_old]
ho_list = [x/16 for x in ho_old]

k = 10

In [20]:
def get_max_degree(arr):
    G_nx = nx.from_numpy_array(arr)
    degrees = [val for (node, val) in G_nx.degree()]
    return max(degrees)

In [7]:
for vill_no in vill_list:

    # our observed network
    stata_household = pd.read_stata('banerjee_data/datav4.0/Data/2. Demographics and Outcomes/household_characteristics.dta')
    file = 'banerjee_data/datav4.0/Data/1. Network Data/Adjacency Matrices/adj_' + 'allVillageRelationships' +'_HH_vilno_' + str(vill_no) + '.csv'
    vill_mat = (pd.read_csv(file, header=None)).to_numpy()
    stata_vill = stata_household.where(stata_household['village'] == vill_no).dropna(how = 'all')
    room_type = np.array(stata_vill['room_no']/np.sqrt((stata_vill['bed_no']+1))<=2)
    room_type_dict = {k: int(v)*2 -1 for k, v in enumerate(room_type)}
    
    kappa_vill = get_max_degree(vill_mat)
    #print(kappa_vill)

    best_sc = -1
    best_ho = -1
    best_emd = np.inf

    for sc in sc_list:
        for ho in ho_list:
            data_dir = 'finer_results_ntwk'
            filename = '{odir}/{vill_no}_{k}_{sc}_{ho}_losses.txt'.format(
                odir=data_dir, vill_no=str(vill_no), k = str(k), sc = str(sc), ho = str(ho))

            f = open(filename, "r")

            # nonsense code for reading the simulated networks and attrs
            ntwk = f.readline()
            attrs = f.readline()
            ntwk = ntwk.replace('[', '')
            ntwk = ntwk.replace(']', '')
            ntwk = ntwk.replace("'", '')
            num_nodes = int(np.sqrt((ntwk.count('0.0') + ntwk.count('1.0'))/5))
            ntwk_arr = np.array([int(float(e)) for e in ntwk.split(',')])
            arr1 = ntwk_arr[:num_nodes**2].reshape((num_nodes, num_nodes))
            arr2 = ntwk_arr[num_nodes**2:2*num_nodes**2].reshape((num_nodes, num_nodes))
            arr3 = ntwk_arr[2*num_nodes**2:3*num_nodes**2].reshape((num_nodes, num_nodes))
            arr4 = ntwk_arr[3*num_nodes**2:4*num_nodes**2].reshape((num_nodes, num_nodes))
            arr5 = ntwk_arr[4*num_nodes**2:5*num_nodes**2].reshape((num_nodes, num_nodes))

            attrs = attrs.strip('{}\n')
            pairs = attrs.split(',')
            attr_dict = {int(key): int(value) for key, value in (pair.split(': ') for pair in pairs)}
            
            sim_dists = [ calc_dists(a, attr_dict, k) for a in [arr1, arr2, arr3, arr4, arr5] ]
            avg_sim_dists = avg_dists(sim_dists)
            vill_dist = calc_dists(vill_mat, room_type_dict, kappa_vill)
            
            sim_tri_count = np.mean([ triangle_count(np.array(a)) for a in [arr1, arr2, arr3, arr4, arr5] ])
            sim_assort = np.mean([ assortativity_coeff(np.array(a), attr_dict) for a in [arr1, arr2, arr3, arr4, arr5] ])
            
            vill_tri_count = triangle_count(vill_mat)
            vill_assort = assortativity_coeff(vill_mat, room_type_dict)
            
            
            curr_emd = ntwk_metric(avg_sim_dists, sim_tri_count, sim_assort, vill_dist, vill_tri_count, vill_assort)

            if curr_emd < best_emd:
                best_emd = curr_emd
                best_sc = sc
                best_ho = ho
                
    print('The best emd for village ', str(vill_no), ' is ', str(best_emd), ' for sc and ho ', str(best_sc), ', ', str(best_ho))


KeyboardInterrupt: 

In [ ]:
#The best emd for village  10  is  0.13435163523262617  for sc and ho  0.0 ,  0.4375

In [21]:
for vill_no in vill_list:
    
    # let's try a different edge set

    stata_household = pd.read_stata('banerjee_data/datav4.0/Data/2. Demographics and Outcomes/household_characteristics.dta')

    # various edge sets
    #money_hyp_files = ['borrowmoney', 'lendmoney', 'keroricecome', 'keroricego']
    #trust_hyp_files = ['helpdecision', 'giveadvice', 'medic']
    trust_fact_files = ['visitcome','visitgo', 'templecompany', 'helpdecision', 'giveadvice', 'medic']


    # choose village and label with normalized room type
    stata_vill = stata_household.where(stata_household['village'] == vill_no).dropna(how = 'all')
    room_type = np.array(stata_vill['room_no']/np.sqrt((stata_vill['bed_no']+1))<=2)
    room_type_dict = {k: v for k, v in enumerate(room_type)}

    old_file3 = 'banerjee_data/datav4.0/Data/1. Network Data/Adjacency Matrices/adj_' + trust_fact_files[0] +'_HH_vilno_' + str(vill_no) + '.csv'
    ad_mat_old3 = (pd.read_csv(old_file3, header=None)).to_numpy()

    for file in trust_fact_files[1:]:
        new_file3 = 'banerjee_data/datav4.0/Data/1. Network Data/Adjacency Matrices/adj_' + file +'_HH_vilno_' + str(vill_no) + '.csv'
        ad_mat_new3= (pd.read_csv(new_file3, header=None)).to_numpy()

        ad_mat_old3 = np.bitwise_or(ad_mat_old3, ad_mat_new3)

    vill_mat = ad_mat_old3
    
    kappa_vill = get_max_degree(vill_mat)
    #print(kappa_vill)

    best_sc = -1
    best_ho = -1
    best_emd = np.inf

    
    best_sc = -1
    best_ho = -1
    best_emd = np.inf

    for sc in sc_list:
        for ho in ho_list:
            data_dir = 'finer_results_ntwk'
            filename = '{odir}/{vill_no}_{k}_{sc}_{ho}_losses.txt'.format(
                odir=data_dir, vill_no=str(vill_no), k = str(k), sc = str(sc), ho = str(ho))

            f = open(filename, "r")

            # nonsense code for reading the simulated networks and attrs
            ntwk = f.readline()
            attrs = f.readline()
            ntwk = ntwk.replace('[', '')
            ntwk = ntwk.replace(']', '')
            ntwk = ntwk.replace("'", '')
            num_nodes = int(np.sqrt((ntwk.count('0.0') + ntwk.count('1.0'))/5))
            ntwk_arr = np.array([int(float(e)) for e in ntwk.split(',')])
            arr1 = ntwk_arr[:num_nodes**2].reshape((num_nodes, num_nodes))
            arr2 = ntwk_arr[num_nodes**2:2*num_nodes**2].reshape((num_nodes, num_nodes))
            arr3 = ntwk_arr[2*num_nodes**2:3*num_nodes**2].reshape((num_nodes, num_nodes))
            arr4 = ntwk_arr[3*num_nodes**2:4*num_nodes**2].reshape((num_nodes, num_nodes))
            arr5 = ntwk_arr[4*num_nodes**2:5*num_nodes**2].reshape((num_nodes, num_nodes))

            attrs = attrs.strip('{}\n')
            pairs = attrs.split(',')
            attr_dict = {int(key): int(value) for key, value in (pair.split(': ') for pair in pairs)}
            
            sim_dists = [ calc_dists(a, attr_dict, k) for a in [arr1, arr2, arr3, arr4, arr5] ]
            avg_sim_dists = avg_dists(sim_dists)
            vill_dist = calc_dists(vill_mat, room_type_dict, kappa_vill)
            
            sim_tri_count = np.mean([ triangle_count(np.array(a)) for a in [arr1, arr2, arr3, arr4, arr5] ])
            sim_assort = np.mean([ assortativity_coeff(np.array(a), attr_dict) for a in [arr1, arr2, arr3, arr4, arr5] ])
            
            vill_tri_count = triangle_count(vill_mat)
            vill_assort = assortativity_coeff(vill_mat, room_type_dict)
            
            
            curr_emd = ntwk_metric(avg_sim_dists, sim_tri_count, sim_assort, vill_dist, vill_tri_count, vill_assort)

            if curr_emd < best_emd:
                best_emd = curr_emd
                best_sc = sc
                best_ho = ho
                
    print('The best emd for village ', str(vill_no), ' is ', str(best_emd), ' for sc and ho ', str(best_sc), ', ', str(best_ho))


The best emd for village  1  is  0.1040894494467567  for sc and ho  0.25 ,  0.3125
The best emd for village  2  is  0.043332661651870455  for sc and ho  0.5625 ,  0.375
The best emd for village  3  is  0.03743375225469291  for sc and ho  0.5 ,  0.375
The best emd for village  4  is  0.02739861425849105  for sc and ho  0.5625 ,  0.4375
The best emd for village  5  is  0.044036242973049256  for sc and ho  0.625 ,  0.625
The best emd for village  6  is  0.041930727839502566  for sc and ho  0.375 ,  0.1875
The best emd for village  7  is  0.11745745126679269  for sc and ho  0.1875 ,  0.375
The best emd for village  8  is  0.04146182463484313  for sc and ho  0.25 ,  0.3125
The best emd for village  9  is  0.12104972125128444  for sc and ho  0.25 ,  0.375
The best emd for village  10  is  0.12282053560450493  for sc and ho  0.25 ,  0.4375
The best emd for village  11  is  0.10822383424135519  for sc and ho  0.1875 ,  0.3125
The best emd for village  12  is  0.08881110981174795  for sc and ho